<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Load-spanish-postal-codes" data-toc-modified-id="Load-spanish-postal-codes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load spanish postal codes</a></span></li><li><span><a href="#Prepare-REGOE-search" data-toc-modified-id="Prepare-REGOE-search-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Prepare REGOE search</a></span><ul class="toc-item"><li><span><a href="#Configure-selenium" data-toc-modified-id="Configure-selenium-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Configure selenium</a></span></li><li><span><a href="#Define-function-to-extract-data-from-results-table" data-toc-modified-id="Define-function-to-extract-data-from-results-table-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Define function to extract data from results table</a></span></li><li><span><a href="#Go-over-postal-codes-list" data-toc-modified-id="Go-over-postal-codes-list-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Go over postal codes list</a></span></li><li><span><a href="#Merge-the-results-and-save" data-toc-modified-id="Merge-the-results-and-save-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Merge the results and save</a></span></li></ul></li></ul></div>

### Import libraries

In [1]:
import time

In [2]:
import tqdm

In [3]:
import requests

In [4]:
from bs4 import BeautifulSoup

In [5]:
import pandas as pd

### Load spanish postal codes

You can find the list here https://códigospostales.es/listado-de-codigos-postales-de-espana/

In [6]:
cp = pd.read_csv('./data/listado-de-codigos-postales-de-españa.csv', 
                 encoding='latin-1', sep=';',
                 dtype={'codigopostalid': object})

In [7]:
cp.head()

,provincia,poblacion,codigopostalid
0,Araba/Álava,Alegría-Dulantzi,01240
1,Ávila,Candeleda,05480
2,Araba/Álava,Vitoria-Gasteiz,01001
3,Araba/Álava,Vitoria-Gasteiz,01002
4,Araba/Álava,Vitoria-Gasteiz,01003


### Prepare REGOE search

#### Configure selenium
- Check Web Scraping using Selenium and Python [tutorial](https://www.scrapingbee.com/blog/selenium-python/)

In [8]:
from selenium import webdriver
import geckodriver_autoinstaller



geckodriver_autoinstaller.install()  # Check if the current version of geckodriver exists
                                     # and if it doesn't exist, download it automatically,
                                     # then add geckodriver to path

driver = webdriver.Firefox()

In [9]:
base_url = 'https://servicio.mapama.gob.es/regoe/Publica/Operadores2.aspx'

#### Define function to extract data from results table

In [10]:
def get_rows(n):
    tabla = driver.find_elements_by_id("CphCuerpoPagina_GVCertificados")
    rows = tabla[0].find_elements_by_xpath("//*[@class='GridViewRowEsilo' or @class='GridViewAlternatingRowEstilo']")
    df = pd.DataFrame([[td.text for td in row.find_elements_by_xpath(".//td")] for row in rows])
    try:
        links = tabla[0].find_element_by_class_name("GridViewPaginacion").find_elements_by_xpath(".//a")
        num = len(links)
        links[n].click()
    except:
        pass
        
    return df

#### Go over postal codes list

In [11]:
total_pages = 0
total_records = 0
pages = []
for cod in tqdm.tqdm(cp['codigopostalid'].unique()):
    driver.get(base_url)
    cp_input = driver.find_element_by_name("ctl00$CphCuerpoPagina$txtCodigoPostal")
    cp_input.send_keys(cod)
    driver.find_element_by_id("CphCuerpoPagina_btnBuscar").click()
    # time.sleep(1)
    tabla = driver.find_elements_by_id("CphCuerpoPagina_GVCertificados")
    if tabla:
        
        try:
            links = tabla[0].find_element_by_class_name("GridViewPaginacion").find_elements_by_xpath(".//a")
            num = len(links)
        except:
            num = 1
           
        for i in range(num):
            df = get_rows(i)
            df['cp'] = cod
            pages.append(df)
            
        total_pages +=num
        print(f'cp:{cod}: {num} pages')

print(f'{total_pages} pages')



  0%|                                      | 2/11039 [00:11<17:44:41,  5.79s/it]

cp:05480: 1 pages


  0%|                                      | 15/11039 [00:37<5:52:46,  1.92s/it]

cp:01015: 1 pages


  0%|▏                                     | 50/11039 [01:45<6:50:37,  2.24s/it]

...

cp:50840: 1 pages


100%|██████████████████████████████████▉| 11027/11039 [8:13:09<00:29,  2.43s/it]

cp:51001: 1 pages


100%|███████████████████████████████████| 11039/11039 [8:13:33<00:00,  2.68s/it]

3463 pages


#### Merge the results and save

In [12]:
df = pd.concat(pages)

In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9,cp
0,,,"FRUTOS DEL BOSQUE DE TIETAR SL,0","ROSALIA DE CASTRO, 3 BAJO-A",Ávila,Castilla y León,Productor agrícola,Castilla y León,"Vegetales y productos vegetales,",SOHISCERT S.A. ES-ECO-002-CL,05480
0,,,"SEMILLAS CLEMENTE, S.A.","C/ ZORROLLETA 3, POL. IND. JUNDIZ",Araba/Álava,País Vasco,Productor agrícola,Comunitat Valenciana,"Vegetales y productos vegetales,",COMITE DE AGRICULTURA ECOLOGICA DE LA COMUNIDA...,01015
0,,,FERNANDO MARTÍNEZ PASCUAL DE LA LLANA,"C/MATARREDO, 29 (BODEGAS HEREDAD DE ADUNA, S.C.)",Araba/Álava,País Vasco,Productor agrícola,"Rioja, La","Vegetales y productos vegetales,",DIRECCIÓN GENERAL DE AGRICULTURA Y GANADERIA E...,01307
1,,,"BERARTE VIÑEDOS Y BODEGAS, S.L.","C/MAYOR, 37",Araba/Álava,País Vasco,Productor agrícola,"Rioja, La","Vegetales y productos vegetales,",DIRECCIÓN GENERAL DE AGRICULTURA Y GANADERIA E...,01307
0,,,"Compañía de vinos Telmo Rodríguez, S.L.","El Monte, s/n",Araba/Álava,País Vasco,"Elaborador / transformador, Mayorista, Product...","Rioja, La, Andalucía, Castilla y León","Productos agroalimentarios, Vegetales y produc...",Consejo de la producción agraria ecológica de ...,01308


In [14]:
df.drop([0,1], axis=1, inplace=True)

In [15]:
df.columns = ['Razón social','Dirección sede social','Provincia','Comunidad Autónoma','Actividad','Ubicación actividad','Grupo de productos','Entidad de control','Codigo postal']

In [16]:
df.to_csv('REGOE_2021.csv')

In [17]:
len(df)

13163

In [18]:
len(pages)

3463